#### Libraries

In [ ]:
import pandas as pd #tabele
import numpy as np #np. wektory, listy
import matplotlib.pyplot as plt #wykresy
import seaborn as sns #wykresy korelacji
from scipy import stats #kruskal-wallis, mann-whitney
from sklearn.impute import KNNImputer
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.tree import export_graphviz
import graphviz

#### Peprocessing

In [ ]:
df0=pd.read_excel("data.xlsx", sheet_name="Arkusz1")
df0=df0.pivot_table(index=("PACJENT_NR", "BADANIE_NR", "ZGON"), columns=["KOD_BADANIA"], values=["WYNIK"]) #ustawienie wyników w kolumnach
df0.reset_index(inplace=True) #rozpakowanie indeksów
df0.columns.name="Lp"
df0.columns=['PACJENT_NR','BADANIE_NR', 'ZGON', 'BETET', 'CO2TET', 'HCO3ACTE', 'HCO3STTE', 'O2SATTET', 'O2TET', 'PHTET']
#zamiana PH na liczbę jonów wodoru, min:35 , max: 45 (im więcej, tym niższe PH)
df0["IONH"]=10**(9-df0["PHTET"])

In [ ]:
#wypełnianie braków metodą K najbliższych sąsiadów
imputer = KNNImputer(n_neighbors=3)
df=df0.copy()
numeric_cols = df.columns[~df.columns.isin(["ZGON"])]
transforms=imputer.fit_transform(df.loc[:, numeric_cols])
df.loc[:, numeric_cols] = transforms
df.head()

In [ ]:
#wybór pacjentów, którzy mają przynajmniej 6 wyników
id_df=df[["PACJENT_NR","BADANIE_NR"]].groupby('PACJENT_NR').max().add_suffix('_liczba')>6
id6=id_df.index[id_df["BADANIE_NR_liczba"]]
df6=df[df['PACJENT_NR'].isin(id6)]

##### Correlation map

In [ ]:
cmap= sns.blend_palette(["#6F92F2","white", "#FF6666"], n_colors=10)
df_corr=df6[[
 'BETET',
 'CO2TET',
 'HCO3ACTE',
 'HCO3STTE',
 'O2SATTET',
 'O2TET','PHTET',
 'IONH']].corr(method='spearman')

df_corr_rounded = df_corr.round(2)
mask = ~np.tril(np.ones(df_corr.shape[1])).astype(bool)

fig, ax = plt.subplots(figsize=(10,8))
fig=sns.heatmap(df_corr_rounded, cmap=cmap, annot=True, mask=mask, linewidths=0.1, annot_kws=dict(size=14))
ax.set_xticklabels(["BE","pCO$_2$", "act. \n HCO$_3$ ", "std. \n  HCO$_3$", "O$_2$ sat." ,"pO$_2$", "pH","[H$^\plus$]" ])
ax.set_yticklabels(["BE","pCO$_2$", "act. \n HCO$_3$ ", "std. \n  HCO$_3$", "O$_2$ sat." ,"pO$_2$", "pH","[H$^\plus$]" ])
sns.set()
plt.xticks(size=12)
plt.yticks(size=12)
plt.xticks(rotation=0)

##### Unification od data

In [ ]:
#normalizacja: 0-min, 1-max
df1=df6.copy()
df1["BETET"] = (df1["BETET"] - (-2.3))/(2.3 - -(2.3))
df1["IONH"] = (df1["IONH"] - 35)/(45 - 35)
df1["O2SATTET"] = (df1["O2SATTET"] - 95)/(100 - 95)
df1["CO2TET"] = (df1["CO2TET"] - 35)/(45 - 35)
df1["O2TET"] = (df1["O2TET"] - 75)/(100 - 75)
df1["HCO3STTE"] = (df1["HCO3STTE"] - 22)/(28 - 22)
df1["HCO3ACTE"] = (df1["HCO3ACTE"] - 22)/(28 - 22)
df1["ZGON"] = np.where(df1["ZGON"] == "NIE", 0, 1)
df1["PHTET"] = (df1["PHTET"] - 7.35)/(7.45 - 7.35)
yeslist = ["PACJENT_NR","BETET", "IONH", "O2SATTET",  "CO2TET"] #"HCO3STTE", "HCO3ACTE", "O2TET"

##### Boxplots

In [ ]:
df_catplot=df1[[ "BETET", "CO2TET", "O2SATTET", "IONH","ZGON"]].copy()
df_catplot["ZGON"].replace({0: "recovered", 1: "deceased"}, inplace=True)
df_transformed = pd.melt(df_catplot, id_vars=['ZGON'], var_name='variable', value_name='value')

plt.figure(figsize=(10, 8))
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
plt.axhline(y=1, color='black', linestyle='--', linewidth=0.5)
sns.boxplot(data=df_transformed, x='variable', y='value', hue="ZGON", palette={'recovered': "#004AAD", 'deceased': "#FF3131"},
            fliersize=3, boxprops=dict(linewidth=1), meanprops=dict(linewidth=0.1))
sns.set(style="whitegrid")

plt.xlabel('Variable')
plt.ylabel('Rescaled value')
plt.yticks(ticks=[-10, -5, 0,  1, 5, 10], labels=[-10, -5, 0,  1, 5, 10])
plt.xticks(ticks=[0,1,2,3,], labels=["BE","pCO$_2$", "O$_2$ sat.", "[H$^\plus$]"])
plt.legend(title="")

##### Welch's test for firsts results

In [ ]:
from scipy.stats import ttest_ind
df6[df6["BADANIE_NR"]==1].groupby("ZGON").mean()
df6[df6["BADANIE_NR"]==1].groupby("ZGON").std()

group1 = df6[((df6["BADANIE_NR"]==1) & (df6["ZGON"]=="TAK"))]
group2 = df6[((df6["BADANIE_NR"]==1) & (df6["ZGON"]=="NIE"))]

#perform Welch's t-test
print(ttest_ind(group1['BETET'], group2['BETET']))
print(ttest_ind(group1['IONH'], group2['IONH']))
print(ttest_ind(group1['CO2TET'], group2['CO2TET']))
print(ttest_ind(group1['O2SATTET'], group2['O2SATTET']))

#### Charts of mean value + std deviation

In [ ]:
df_line=df6[[ "BADANIE_NR", "IONH","ZGON"]].copy()
df_line["ZGON"].replace({"NIE": "recovered", "TAK": "deceased"}, inplace=True)
# sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
fig1=sns.lineplot(data=df_line, x="BADANIE_NR", y='IONH', hue="ZGON", ci='sd', palette={'recovered': "#6F92F2", 'deceased': "#FF6666"}),

plt.xlabel('measurement number')
plt.ylabel('[H$^{\plus}$]')
plt.xlim(0,50)
plt.xticks(ticks=[1,10,20,30,40,50])
plt.legend(title="")


In [ ]:
df_line=df6[[ "BADANIE_NR", "CO2TET","ZGON"]].copy()
df_line["ZGON"].replace({"NIE": "recovered", "TAK": "deceased"}, inplace=True)
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
fig2=sns.lineplot(data=df_line, x="BADANIE_NR", y='CO2TET', hue="ZGON", ci='sd', palette={'recovered': "#6F92F2", 'deceased': "#FF6666"})

plt.xlabel('measurement number')
plt.ylabel('pCO$_2$')
plt.xlim(0,50)
plt.xticks(ticks=[1,10,20,30,40,50])
plt.legend(title="")

In [ ]:
df_line=df6[[ "BADANIE_NR", "O2SATTET","ZGON"]].copy()
df_line["ZGON"].replace({"NIE": "recovered", "TAK": "deceased"}, inplace=True)
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
fig3=sns.lineplot(data=df_line, x="BADANIE_NR", y='O2SATTET', hue="ZGON", ci='sd', palette={'recovered': "#6F92F2", 'deceased': "#FF6666"})

plt.xlabel('measurement number')
plt.ylabel('O$_2$ saturation')
plt.xlim(0,50)
plt.xticks(ticks=[1,10,20,30,40,50])
plt.legend(title="")

In [ ]:
df_line=df6[[ "BADANIE_NR", "BETET","ZGON"]].copy()
df_line["ZGON"].replace({"NIE": "recovered", "TAK": "deceased"}, inplace=True)
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
fig4=sns.lineplot(data=df_line, x="BADANIE_NR", y='BETET', hue="ZGON", ci='sd', palette={'recovered': "#6F92F2", 'deceased': "#FF6666"})

plt.xlabel('measurement number')
plt.ylabel('BE')
plt.xlim(0,50)
plt.xticks(ticks=[1,10,20,30,40,50])
plt.legend(title="")

#### Features extraction

In [ ]:
from itertools import groupby
def rle(data):
  x = [len(list(y)) for x, y in groupby(data) if x==True]
  if x==[]:
    return 0
  return max(x)

In [ ]:
#Obliczenie różnic
dfr=df.copy()
nr_pacjenta=dfr["PACJENT_NR"]
dfr2=dfr[yeslist].groupby('PACJENT_NR').diff()
dfr3=pd.concat([nr_pacjenta, dfr2], axis=1)
dfr_mdiff=dfr3.groupby('PACJENT_NR').mean().add_suffix('_mdiff')
#dfr_mdiff


In [ ]:
df_mean = df[yeslist].groupby('PACJENT_NR').mean().add_suffix('_mean')
df_min = df[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max = df[yeslist].groupby('PACJENT_NR').max().add_suffix('max')
df_med = df[yeslist].groupby('PACJENT_NR').median().add_suffix('median')
df_size = df[['PACJENT_NR', 'BADANIE_NR']].groupby('PACJENT_NR').count().add_suffix('_size')
df_std = df[yeslist].groupby('PACJENT_NR').std().add_suffix('_std')
df_sum = df[yeslist].groupby('PACJENT_NR').sum().add_suffix('_sum')

df_rzad_pom_ups=pd.concat([df1["PACJENT_NR"],df1[["BETET", "IONH", "O2SATTET", "CO2TET"]]>1], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_ups=df_rzad_pom_ups.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_ups')

df_rzad_pom_downs=pd.concat([df1["PACJENT_NR"],df1[["BETET", "IONH", "O2SATTET", "CO2TET"]]<0], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_downs=df_rzad_pom_downs.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_downs')

df_zgon = df1[['PACJENT_NR', 'ZGON']].groupby('PACJENT_NR').first()


#Zliczanie wyników za niskich
df_no_drops = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_drops[col] = np.where(df1[col] < 0, 1, 0) #1-za niski, 0-nie za niski

#Zliczanie wyników za wysokich
df_no_ups = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_ups[col] = np.where(df1[col] > 1, 1, 0) #1-za wysoki, 0-nie za wysoki

#Zliczanie wyników poza normą drop-poniżej up-powyżej
df_no_dropup = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR": #dla innych kolumn niż numer pacjenta
        df_no_dropup[col] = df_no_ups[col] + df_no_drops[col]

#Dodanie kolumny nOK zliczającej, ile wyników pacjent miał ok
df_no_dropup["nOK"] = 0
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_dropup["nOK"] += df_no_dropup[col]

#Dodanie kolumny sOK, która przyjmuje wartość 1, gdy wszystkie wyniki były ok
df_no_dropup["sOK"] = np.where(df_no_dropup["nOK"] == 0, 1, 0)

#wypisz tablicę wszystkich wyników
#print(df_no_dropup)

#agregacja
df_no_drops = df_no_drops[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_drops')
df_no_ups = df_no_ups[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_up')
df_no_dropup = df_no_dropup[["PACJENT_NR","BETET", "IONH", "O2SATTET", "CO2TET", "nOK", "sOK"]].groupby('PACJENT_NR').sum().add_suffix('_no_dropup')

#"O2TET", "HCO3STTE", "HCO3ACTE"

#łącznie
df_list = [df_size,df_zgon, df_min, df_max, df_no_drops, df_no_ups, df_no_dropup, dfr_mdiff, df_rzad_ups, df_rzad_downs]
#'df_mean, df_med, df_std, df_sum'
df_all = pd.concat(df_list, axis=1)

#usuwanie kolumn, które nie będą używane
df_all.drop(columns=["O2SATTET_no_up", "O2SATTET_no_dropup", "O2SATTET_rzad_ups" ,  "O2SATTETmax"], inplace=True)

In [ ]:
#srednia wynikow min i max w grupach
yeslist = ["PACJENT_NR","BETET", "IONH", "O2SATTET",  "CO2TET", "HCO3STTE", "HCO3ACTE", "O2TET", "PHTET"]
df_min2 = df6[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max2 = df6[yeslist].groupby('PACJENT_NR').max().add_suffix('max')

df_list2 = [df_zgon, df_size, df_min2, df_max2 ]
df_min_max = pd.concat(df_list2, axis=1)

df_min_max.head()

min_max_mean = df_min_max.groupby('ZGON').mean()
print(min_max_mean)

#### Selecting important variables

In [ ]:
df_all = df_all[df_all["BADANIE_NR_size"] > 6]
df_all["ZGON"].replace({0: "recovered", 1: "deceased"}, inplace=True)
df2_dead = df_all[df_all["ZGON"]=="deceased"].copy()
df2_life = df_all[df_all["ZGON"]=="recovered"].copy()

df2_dead.drop(columns=["ZGON"], inplace=True)
df2_life.drop(columns=["ZGON"], inplace=True)
df2_dead.columns

In [ ]:
#metoda Manna-Whitneya
tabela_MN=pd.DataFrame(columns=("nazwa", "statystyka", "p-wartosc"))
for column in df2_dead.columns:
    kw = stats.mannwhitneyu(df2_dead[column], df2_life[column])
    new_row = {"nazwa": column, "statystyka": kw[0], "p-wartosc": kw[1]}
    tabela_MN = pd.concat([tabela_MN, pd.DataFrame([new_row])], ignore_index=True)
nazwy_sign=tabela_MN[tabela_MN["p-wartosc"]<0.05].nazwa

#### Classification

##### Dummmy classificator

In [ ]:
#dummy classifier
X=df_all[nazwy_sign]
y=df_all["ZGON"]

model=DummyClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=18)

s=list()
stren=list()
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  s.append(np.round(model.score(X.iloc[test_index], y.iloc[test_index]),4))

print("Wyniki kroswalidacji: ", s)
print("Dokładność drzewa: ", np.mean(s))

##### Decision Tree

In [ ]:
#drzewo decyzyjne z walidacją krzyżową i zbalansowaniem klas przy podziale na treningowe i testowe
X=df_all[nazwy_sign]
y=df_all["ZGON"]
model = tree.DecisionTreeClassifier(random_state=5, max_depth=3)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=18) #5

s=list()
stren=list()
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  s.append(np.round(model.score(X.iloc[test_index], y.iloc[test_index]),4))
  stren.append(np.round(model.score(X.iloc[train_index], y.iloc[train_index]),4))

print("Wyniki uczenia: ", stren)
print("Dokładność uczenia: ", np.mean(stren))
print()
print("Wyniki kroswalidacji: ", s)
print("Dokładność drzewa: ", np.mean(s))

In [ ]:
model.fit(X, y)

##### Feature importances

In [ ]:
wartosci=model.feature_importances_
print(X.columns)
kategorie=['minimum BETET \n measurement', 'minimum [H$^\plus$] \n (maximum pH)', 'maximum [H$^\plus$] (minimum pH)', 'maximum pCO$_2$', 'BETET below the norm [number of cases]',
           '[H$^\plus$] below the norm \n (pH above the norm [number of cases])', 'pO$_2$ saturation below the norm \n [number of cases]', 'pCO$_2$ below the norm [number of cases]',
           'BETET above the norm [number of cases]',
           '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases]', 'average change in \n ion H concentration', 'average change in \n O$_2$ saturation', 'average change in pCO$_2$',
           'BETET above the norm [number of cases in a row]', '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases in a row]', 'BETET below the norm [number of cases in a row]',
           '[H$^\plus$] below the norm \n (pH above the norm) \n [number of cases in a row]', 'O$_2$ saturation above the norm \n [number of cases in a row]']

In [ ]:
# Tworzenie wykresu słupkowego
niezerowe_wartosci = [x for x in wartosci if x != 0]
niezerowe_kategorie = [kategorie[i] for i, x in enumerate(wartosci) if x != 0]

#sortowanie
sorted_niezerowe_wartosci, sorted_niezerowe_kategorie = zip(*sorted(zip(niezerowe_wartosci, niezerowe_kategorie), reverse=False))

# Tworzenie wykresu słupkowego z niezerowymi wartościami
fig, ax = plt.subplots(figsize=(7,5))
plt.barh(sorted_niezerowe_kategorie, sorted_niezerowe_wartosci)

plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
ax.yaxis.set_label_coords(-0.5, 0.5)
plt.show()

#### Tree visualisation

In [ ]:
dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=[ "ZGON - TAK","ZGON - NIE"],
                           filled=False, rounded=True)

graph = graphviz.Source(dot_data)
graph


In [ ]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=["ZGON - NIE", "ZGON - TAK"],
                           filled=False, rounded=True)
  graph = graphviz.Source(dot_data)
  graph.format = 'png'
  graph.render(f'drzewo_gazometria_{i}')